In [123]:
import pandas as pd

In [124]:
df = pd.read_csv('../input/senex-mlp/data_frame_pós_ETL.csv')
df.drop(columns=['Unnamed: 0','ESTADO','ANO'], inplace = True)

In [125]:
df

,IDADE,SEXO,RACACOR,ESTCIV,CAUSABAS
0,91,1,1,2,C61
1,80,1,1,2,I500
2,79,2,1,3,I500
3,63,1,1,2,C710
4,68,2,1,2,C349
...,...,...,...,...,...
208526,83,2,1,3,I429
208527,66,1,4,1,K769
208528,84,1,1,2,K921
208529,85,2,1,3,I219


In [126]:
df_teste = df.iloc[0:10001]
df_teste

,IDADE,SEXO,RACACOR,ESTCIV,CAUSABAS
0,91,1,1,2,C61
1,80,1,1,2,I500
2,79,2,1,3,I500
3,63,1,1,2,C710
4,68,2,1,2,C349
...,...,...,...,...,...
9996,74,2,1,3,C260
9997,61,1,3,2,C169
9998,71,1,2,2,I10
9999,78,2,1,3,C169


In [127]:
X = df_teste.iloc[:, :-1].values
y = pd.DataFrame(df_teste.iloc[:, 4].values,columns=['CAUSABAS'])

y

,CAUSABAS
0,C61
1,I500
2,I500
3,C710
4,C349
...,...
9996,C260
9997,C169
9998,I10
9999,C169


In [128]:
X = df_teste.iloc[:, 0:-1].values #features: da primeira até a penúltima coluna
#y = df_teste.iloc[:, -1].values #rótulo: última coluna
y = pd.DataFrame(df_teste.iloc[:, -1].values,columns=['CAUSABAS'])

In [129]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = y.apply(le.fit_transform)
y

,CAUSABAS
0,139
1,402
2,402
3,147
4,99
...,...
9996,88
9997,60
9998,347
9999,60


In [130]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 1/3, 
                                                    random_state = 42)

In [131]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train,
                                                      y_train, 
                                                      test_size = 1/3, 
                                                      random_state = 42)

In [132]:
# Feature Scaling: alguns algoritmos são sensíveis à escala
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_2 = scaler.fit_transform(X_train_2)
X_val = scaler.transform(X_val)

In [133]:
#Dicionário de resultados
dic = {}

In [134]:
from sklearn.metrics import accuracy_score

In [135]:
#regressão logistica
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 42)
lr.fit(X_train_2, y_train_2)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=42)

In [136]:
# Treinando o k-NN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train_2, y_train_2)

/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=7)

In [137]:
# Avaliando o k-NN e anotando no dicionário
dic["kNN_7"] =  accuracy_score(y_val, knn.predict(X_val))

In [138]:
#Treinando uma árvore de decisão
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 50, random_state = 42)
tree.fit(X_train_2, y_train_2)

DecisionTreeClassifier(max_depth=50, random_state=42)

In [139]:
# Avaliando a árvore e anotando no dicionário
dic["Decision_tree_4"] = accuracy_score(y_val, tree.predict(X_val))

In [140]:
#Treinando uma floresta aleatória
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators= 100, 
                            random_state = 42)

In [141]:
# Avaliando a florest aleatória e anotando no dicionário
rf.fit(X_train_2, y_train_2)
dic["Random_forest_100"] = accuracy_score(y_val, rf.predict(X_val))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [142]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
#Avaliando o support vector machine e anotando no dicionário
svc.fit(X_train_2, y_train_2)
dic["SVC"] = accuracy_score(y_val, svc.predict(X_val))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [143]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(3,3,3,3),max_iter=3000,random_state=42)
mlp.fit(X_train_2, y_train_2)
dic["mlp"] = accuracy_score(y_val, mlp.predict(X_val))

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [144]:
validation = pd.Series(dic, name="Acurácia")

In [145]:
validation

kNN_7                0.026991
Decision_tree_4      0.053081
Random_forest_100    0.050832
SVC                  0.086370
mlp                  0.077373
Name: Acurácia, dtype: float64

In [146]:
#Treinando o melhor modelo no conjunto inteiro:
scaler = MinMaxScaler()    
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  

In [147]:
dt = DecisionTreeClassifier(random_state = 42)     
dt.fit(X_train, y_train) 

DecisionTreeClassifier(random_state=42)

In [149]:
print("Acurácia final: ", str(accuracy_score(y_test, dt.predict(X_test))))

Acurácia final:  0.05158968206358728


Como as máquinas performaram de maneira horrível, será necessário diminuir as possibilidades de desfecho. Sugestão: 0 -> doenças comuns, 1-> menos comuns, 2 -> raras
Para isso será necessário separar as CID's em grupos mais frequentes, menos frequentes e raras.